In [10]:
import random
import math
import csv

# =======================================Filename Of Utilzed Files====================================
TRAINING_FILENAME = "Training_File"
VALIDATION_FILENAME = "Validation_File"

#======================================================================================================
#========================================Class Neuron==================================================
#======================================================================================================


class Neuron():
    def __init__(self, Activation_Value, Num_of_Weights, Neuron_Index):
        self.AV = Activation_Value
        self.Weights = []
        self.Neuron_Index = Neuron_Index
        self.Gradiant_Value = 0
        self.Delta_Weights = []
        for j in range(Num_of_Weights):
            self.Weights.append(random.random())
        for k in range(Num_of_Weights):
            self.Delta_Weights.append(0)

    def Multi_Weights(self, Prev_Layer):  # Calculating Values for next Following Neuron

        result = 0

        for j in range(len(Prev_Layer)):
            result = Prev_Layer[j].AV * Prev_Layer[j].Weights[self.Neuron_Index] + result

        return 1 / (1 + math.exp(-Lamda*result))

#======================================================================================================
#======================================================================================================
#======================================================================================================




Input_Layer  = [Neuron(0, 4, 0), Neuron(0, 4, 1),Neuron(1, 4, 2)]
Hidden_Layer = [Neuron(0, 2, 0), Neuron(0, 2, 1),Neuron(0, 2, 2),Neuron(0, 2, 3),Neuron(1, 2, 4)]
Output_Layer = [Neuron(0, 0, 0), Neuron(0, 0, 1)]

#    #Bias Neurons for both input and hidden layer
#    Input_Layer[2].AV = 1
#    Hidden_Layer[2].AV = 1

# Lamda = 0.05
# lr = 0.10
# mt = 0.15

#======================================================================================================
#=======================================Declared Constants============================================= 
#======================================================================================================
Lamda = 0.15
lr    = 0.6
mt    = 0.2

#======================================================================================================
#=====================================Feed Forward Process============================================= 
#======================================================================================================

def FeedForward_process(inputs):
    for i in range(len(Input_Layer) - 1):
        Input_Layer[i].AV = inputs[i]

    for i in range(len(Hidden_Layer) - 1):
        Hidden_Layer[i].AV = Hidden_Layer[i].Multi_Weights(Input_Layer)

    for i in range(len(Output_Layer)):
        Output_Layer[i].AV = Output_Layer[i].Multi_Weights(Hidden_Layer)

    Feed_Forward_Output = []

    for i in range(len(Output_Layer)):
        Feed_Forward_Output.append(Output_Layer[i].AV)

    return Feed_Forward_Output

#======================================================================================================
#=========================================Back Propagation Process=====================================
#======================================================================================================

def BackPropagation_process(outputs):
    error = []
    for i in range(len(Output_Layer)):
        error.append(0)

    for i in range(len(Output_Layer)):
        error[i] = outputs[i] - Output_Layer[i].AV

    for i in range(len(Output_Layer)):
        Output_Layer[i].Gradiant_Value = Lamda * Output_Layer[i].AV * (1 - Output_Layer[i].AV) * error[i]

    for i in range(len(Hidden_Layer)):
        SUM = 0

        for j in range(len(Output_Layer)):
            SUM = Output_Layer[j].Gradiant_Value * Hidden_Layer[i].Weights[j] + SUM

        Hidden_Layer[i].Gradiant_Value = Lamda * Hidden_Layer[i].AV * (1 - Hidden_Layer[i].AV) * SUM

    for i in range(len(Hidden_Layer)):
        for j in range(len(Output_Layer)):
            Hidden_Layer[i].Delta_Weights[j] = lr * Output_Layer[j].Gradiant_Value * Hidden_Layer[i].AV + mt * \
                                               Hidden_Layer[i].Delta_Weights[j]

    for i in range(len(Input_Layer)):
        for j in range(len(Hidden_Layer) - 1):
            Input_Layer[i].Delta_Weights[j] = lr * Hidden_Layer[j].Gradiant_Value * Input_Layer[i].AV + mt * \
                                              Input_Layer[i].Delta_Weights[j]

    for i in range(len(Hidden_Layer)):
        for j in range(len(Output_Layer)):
            Hidden_Layer[i].Weights[j] = Hidden_Layer[i].Delta_Weights[j] + Hidden_Layer[i].Weights[j]

    for i in range(len(Input_Layer)):
        for j in range(len(Hidden_Layer) - 1):
            Input_Layer[i].Weights[j] = Input_Layer[i].Delta_Weights[j] + Input_Layer[i].Weights[j]


#======================================================================================================
# ================================================Epoch================================================
#======================================================================================================


def Epoch(Trainigfilename):
    Training_Data_Rows = []
    File = open('%s.csv' % Trainigfilename, "r")
    Aquired_Data = list(csv.reader(File))

    for i in range(len(Aquired_Data)):
        Norm_Inputs = []
        Norm_Expec_Outputs = []

        for j in range(len(Input_Layer) - 1):
            Norm_Inputs.append(float(Aquired_Data[i][j]))
            Norm_Expec_Outputs.append(float(Aquired_Data[i][j + 2]))

        Output = FeedForward_process(Norm_Inputs)
        BackPropagation_process(Norm_Expec_Outputs)

#======================================================================================================
#================================================Training Function=====================================
#======================================================================================================


def Training_Error_Calc(TrainigFilename):
    Training_Data_Rows = []
    File = open('%s.csv' % TrainigFilename, "r")
    Aquired_Data = list(csv.reader(File))

    Training_Error = 0

    for i in range(len(Aquired_Data)):
        Norm_Inputs = []
        Norm_Expec_Outputs = []
        Calc_Output = []

        for j in range(len(Input_Layer) - 1):
            Norm_Inputs.append(float(Aquired_Data[i][j]))
            Norm_Expec_Outputs.append(float(Aquired_Data[i][j + 2]))

        Calc_Output = FeedForward_process(Norm_Inputs)
        Training_Error += (((Calc_Output[0] - Norm_Expec_Outputs[0]) ** 2) + (
                    (Calc_Output[1] - Norm_Expec_Outputs[1]) ** 2)) / len(Calc_Output)

    Training_Error = math.sqrt(Training_Error / len(Aquired_Data))

    return Training_Error

#======================================================================================================
#================================================Validation Function===================================
#======================================================================================================


def Validation_Error_Calc(ValidationFilename):
    Training_Data_Rows = []
    File = open('%s.csv' % ValidationFilename, "r")
    Aquired_Data = list(csv.reader(File))

    Validation_Error = 0

    for i in range(len(Aquired_Data)):
        Norm_Inputs = []
        Norm_Expec_Outputs = []
        Calc_Output = []

        for j in range(len(Input_Layer) - 1):
            Norm_Inputs.append(float(Aquired_Data[i][j]))
            Norm_Expec_Outputs.append(float(Aquired_Data[i][j + 2]))

        Calc_Output = FeedForward_process(Norm_Inputs)
        Validation_Error += (((Calc_Output[0] - Norm_Expec_Outputs[0]) ** 2) + (
                    (Calc_Output[1] - Norm_Expec_Outputs[1]) ** 2)) / len(Calc_Output)

    Validation_Error = math.sqrt(Validation_Error / len(Aquired_Data))

    return Validation_Error

#======================================================================================================
#=================================================Training Process=====================================
#======================================================================================================


Previous_Validation_Error = 0

Validation_Queue = []


for i in range(500):

    print("Epoch " + str(i))
    
    if i > 1:
        if Previous_Validation_Error < ValidationError:
            break
        
    if i != 0:
        Previous_Validation_Error = ValidationError
    
    Epoch(TRAINING_FILENAME)
    TrainingError = Training_Error_Calc(TRAINING_FILENAME)
    ValidationError = Validation_Error_Calc(VALIDATION_FILENAME)
    
    Validation_and_Weights = []
    
    print("Training Error            :" + str(TrainingError))
    print("Previous Validation Error :" + str(Previous_Validation_Error))
    print("Validation Error          :" + str(ValidationError))

    
#======================================================================================================
#Adding the weights and validation errot to a list
#======================================================================================================

    Validation_and_Weights.append(ValidationError)

    for Input_Neuron in Input_Layer:
        for k in range(len(Input_Neuron.Weights)):
            Validation_and_Weights.append(Input_Neuron.Weights[k])

    for Hidden_Neuron in Hidden_Layer:
        for k in range(len(Hidden_Neuron.Weights)):
            Validation_and_Weights.append(Hidden_Neuron.Weights[k])
    
    print(Validation_and_Weights)
#==================================Trying to avoid Local minima==================================================

    Validation_Queue.append(Validation_and_Weights)
    
    
    if len(Validation_Queue) > 5:
        Validation_Queue.pop(0)

    g = 0
    
    Marked_Weight_Index = 0
    
#======================================================================================================


    if (i != 0):
        
        if Previous_Validation_Error < ValidationError:

            Required_Validation_Error = Validation_Queue[0][0]
            Marked_Weight_Index = 0

            for j in range(len(Validation_Queue)):
                if (Validation_Queue[j][0] < Required_Validation_Error):
                    Required_Validation_Error = Validation_Queue[j][0]
                    Marked_Weight_Index = j

            print(Validation_Queue[Marked_Weight_Index][0])


            for Input_Neuron in Input_Layer:
                for j in range(len(Input_Neuron.Weights)):
                    Input_Neuron.Weights[j] = Validation_Queue[Marked_Weight_Index][j + 1]

            for Hidden_Neuron in Hidden_Layer:
                for j in range(len(Hidden_Neuron.Weights)):
                    Hidden_Neuron.Weights[j] = Validation_Queue[Marked_Weight_Index][len(Input_Neuron.Weights) + j + 1]

            Previous_Validation_Error = Validation_Queue[Marked_Weight_Index][0]

            g = 1

    
   

#======================================================================================================

#    if i!= 0:
#
#        if (Previous_Validation_Error - ValidationError) < 0.000001:#
#
#           if g == 0:
#
#                for Input_Neuron in Input_Layer:
#                    for j in range(len(Input_Neuron.Weights)):
#                        Input_Neuron.Weights[j] = Validation_Queue[len(Validation_Queue)-1][j + 1]#
#
#                for Hidden_Neuron in Hidden_Layer:
#                    for j in range(len(Hidden_Neuron.Weights)):
#                        Hidden_Neuron.Weights[j] = Validation_Queue[len(Validation_Queue)-1][len(Input_Neuron.Weights) + j + 1]
#
#            break


#======================================================================================================
#====================Printing final weights into a text file for later reference=======================    
#======================================================================================================


with open('Final_Weights.txt', 'w') as txt_file:
    for Input_Neuron in Input_Layer:

        for i in range(len(Input_Neuron.Weights)):
            txt_file.writelines(str(Input_Neuron.Weights[i]))
            txt_file.writelines(",")
        txt_file.writelines("\n")

    txt_file.writelines("\n")

    for Hidden_Neuron in Hidden_Layer:

        for i in range(len(Hidden_Neuron.Weights)):
            txt_file.writelines(str(Hidden_Neuron.Weights[i]))
            txt_file.writelines(",")
        txt_file.writelines("\n")

#======================================================================================================


Epoch 0
Training Error            :0.1544954026637495
Previous Validation Error :0
Validation Error          :0.14342058973207344
[0.14342058973207344, 0.4391311569508865, 0.16487973210623055, 0.16304623339812155, 0.6343864558477923, 0.7908505595717525, 0.748365266426445, 0.4959017652226183, 0.3827164183446368, 0.8427700233852031, 0.8302764936680959, 0.663180214772723, 0.4705959653009038, 0.23011155317392887, 0.5176523798699505, -0.11992484481398749, 0.9196961305773592, 0.8024448637247125, 0.7277072290324423, 0.6248369367101382, 0.8554605372500275, 0.0025184417373462177, 0.14482097179764253]
Epoch 1
Training Error            :0.1544737326372146
Previous Validation Error :0.14342058973207344
Validation Error          :0.14339350094647288
[0.14339350094647288, 0.4293697460641137, 0.22394901060340028, 0.2383869233247633, 0.7485475726631878, 0.8141458310641779, 0.8082357626655413, 0.37501322312613633, 0.27831303894437914, 0.8295432857012724, 0.8126315381324778, 0.6536394450638713, 0.464714